In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=bec6de1e8a3a9074220a403c0ce188e6abea76573a1a55f1c28c45965c5e9b90
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 5.1MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0257 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 8.05 , C = 787.74 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_244000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_24_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=402125, Sun Apr 19 02:13:30 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 9.411984418203683e+03 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 9.397641150879546e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 9.390943416354770e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9458.21590202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9412.79176634] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   40    760 9.166695838007736e+03 1.1e+00 1.97e+00  2e+00  2e+00 0:03.3
   87   1653 9.005854057402286e+03 1.2e+00 1.53e+00  2e+00  2e+00 0:07.4
  100   1900 8.684385651085699e+03 1.2e+00 1.38e+00  1e+00  1e+00 0:08.4
  171   3249 7.658048362832827e+03 1.2e+00 7.74e-01  8e-01  8e-01 0:14.4
  200   3800 7.072555749617476e+03 1.2e+00 5.76e-01  6e-01  6e-01 0:16.8
  299   5681 6.123148259297253e+03 1.2e+00 2.15e-01  2e-01  2e-01 0:24.9
  300   5700 6.117228438350588e+03 1.2e+00 2.13e-01  2e-01  2e-01 0:24.9
  400   7600 5.981509498994436e+03 1.2e+00 6.76e-02  7e-02  7e-02 0:33.1
  500   9500 5.967811286765518e+03 1.2e+00 2.22e-02  2e-02  2e-02 0:41.3
  600  11400 5.966178351786200e+03 1.3e+00 7.40e-03  7e-03  7e-03 0:49.5
  700  13300 5.966025279598241e+03 1.3e+00 2.62e-03  3e-03  3e-03 0:57.6
  800  15200 5.966003609945527e+03 1.3e+00 8.91e-04  9e-04  9e-04 1:05.8
  900  17100 5.966001165957049e+03 1.3e+00 2.84e-04  3e-04  3e-04 1:13.9
 1000  19000 5.966000909606836e+03 1.3e+00 8.74e-05

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10398.0922062] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.809246428635577e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 9.070025683981639e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3


...

    3     57 9.338740996889042e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   32    608 9.149378207067370e+03 1.1e+00 1.90e+00  2e+00  2e+00 0:03.4
   72   1368 8.828666114580108e+03 1.1e+00 1.62e+00  2e+00  2e+00 0:07.4
  100   1900 8.697293722507518e+03 1.2e+00 1.32e+00  1e+00  1e+00 0:10.3
  159   3021 7.665451784473724e+03 1.2e+00 7.61e-01  8e-01  8e-01 0:16.3
  200   3800 6.854384617102366e+03 1.2e+00 4.80e-01  5e-01  5e-01 0:20.5
  279   5301 6.127779968122560e+03 1.2e+00 2.19e-01  2e-01  2e-01 0:28.6
  300   5700 6.068877605718101e+03 1.2e+00 1.74e-01  2e-01  2e-01 0:30.8
  399   7581 5.978805586911107e+03 1.2e+00 6.01e-02  6e-02  6e-02 0:40.8
  400   7600 5.978437912885370e+03 1.2e+00 5.94e-02  6e-02  6e-02 0:40.9
  500   9500 5.967818086412170e+03 1.2e+00 2.11e-02  2e-02  2e-02 0:51.0
  600  11400 5.966232601938837e+03 1.2e+00 7.78e-03  8e-03  8e-03 1:01.3
  700  13300 5.966029829589433e+03 1.2e+00 2.68e-03  3e-03  3e-03 1:11.3
  800  15200 5.966005133148274e+03 1.2e+00 1.04e-03

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9534.94284712] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.393539503994369e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 9.363985242136614e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9596.49633157] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   37    703 8.037978093818478e+03 1.1e+00 1.84e+00  2e+00  2e+00 0:03.3
   82   1558 6.319841716129244e+03 1.2e+00 1.62e+00  2e+00  2e+00 0:07.4
  100   1900 5.801185505506695e+03 1.2e+00 1.50e+00  1e+00  2e+00 0:09.0
  166   3154 3.463525461473203e+03 1.2e+00 9.46e-01  9e-01  1e+00 0:15.1
  200   3800 2.972425016188788e+03 1.2e+00 6.96e-01  7e-01  7e-01 0:18.2
  287   5453 2.363537539516624e+03 1.2e+00 3.27e-01  3e-01  3e-01 0:26.3
  300   5700 2.309491892675211e+03 1.3e+00 2.90e-01  3e-01  3e-01 0:27.5
  400   7600 2.106042881522179e+03 1.3e+00 1.27e-01  1e-01  1e-01 0:36.6
  500   9500 2.072547552811842e+03 1.3e+00 5.41e-02  5e-02  5e-02 0:45.8
  600  11400 2.066861081017845e+03 1.3e+00 2.47e-02  2e-02  2e-02 0:54.8
  700  13300 2.065626248581205e+03 1.3e+00 1.02e-02  1e-02  1e-02 1:04.1
  800  15200 2.065373719760238e+03 1.3e+00 4.75e-03  5e-03  5e-03 1:13.4
  900  17100 2.065333304008522e+03 1.3e+00 2.04e-03  2e-03  2e-03 1:22.5
 1000  19000 2.065322253385274e+03 1.3e+00 1.01e-03

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F24_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:200])
    SVM_Fun [i] = F24(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F24(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2661.80685184]), count=array([1])) 0.000398124034450725
SVM
ModeResult(mode=array([8757.94073824]), count=array([1])) 722.8359406835918
ELN
ModeResult(mode=array([10437.83556329]), count=array([1])) 0.000208596516750649
